# Data Extraction and Cleaning Workbook

In [413]:
import pandas as pd 
import spotipy 
from operator import itemgetter

#need to add in my own client ID and client credentials
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
cid ="31d1e524a4c045768a7e34cf4f4d622a" 
secret = "a7676e9d2b45454db896372e6f0986a5" 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False 

playlist_ids = []
playlist_names = []


#search first 100 playlists with keyword happy in it and store their ids and names
for i in range(100):
    playlist_results = sp.search(q='happy', type='playlist', limit=1, offset = i)
    playlists = playlist_results["playlists"]["items"] 
    for s in playlists:
        playlist_ids.append((s['id']))
        playlist_names.append((s['name']))
    
#print(playlist_names)
# print(playlist_ids)

count = 0
track_names=[]
playlist_artist_names = []
playlist_artist_ids = []

#for each playlist, store a track's items(track id, name, and artist)
for p in playlist_ids:
    playlist = sp.playlist(p, fields=all, market=None, additional_types=('track',))
    tracks = playlist["tracks"]["items"] 
    track_ids = []

    #Picking first 100 tracks in a playlist to make it compatible with the audio features API pull limit 
    for t in range(100):#len(tracks)):
        try:
            track_ids.append(tracks[t]["track"]["id"]) 
            track_names.append(tracks[t]["track"]["name"])
            track_artists = tracks[t]["track"]["artists"]
            track_artist_names = []
            track_artist_ids = []
            #for each artist, store their name and id
            for s in track_artists:
                track_artist_names.append((s['name']))
                track_artist_ids.append((s['id']))

            playlist_artist_names.append(track_artist_names)
            playlist_artist_ids.append(track_artist_ids)
        except:
            pass

    #100 row limit to getting audio features        
    try:
        features=sp.audio_features(track_ids)
    except:
        pass

    if count == 0:
        total_happy_df = pd.DataFrame(features)
    else:
        features_df = pd.DataFrame(features)
        total_happy_df = total_happy_df.append(features_df, ignore_index = True)

    count = count + 1

#Make my dataframes
total_happy_df['track_names']=track_names
total_happy_df['artist_names'] = playlist_artist_names
total_happy_df['artist_ids'] = playlist_artist_ids

happy_df = total_happy_df.drop_duplicates(subset = 'id')
happy_df['happy'] = 1
happy_df



/Users/isaacnwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,track_names,artist_names,artist_ids,happy
0,0.813,0.640,2,-5.908,1,0.2720,0.2690,0.000000,0.1310,0.488,...,34FbkFgAbv4ffvhL2XQMVm,spotify:track:34FbkFgAbv4ffvhL2XQMVm,https://api.spotify.com/v1/tracks/34FbkFgAbv4f...,https://api.spotify.com/v1/audio-analysis/34Fb...,147076,4,Jolie nana,[Aya Nakamura],[7IlRNXHjoOCgEAWN5qYksg],1
1,0.729,0.788,10,-4.832,0,0.0908,0.0703,0.000000,0.1150,0.857,...,3GYlQmwfBDo7imFyGPhrl7,spotify:track:3GYlQmwfBDo7imFyGPhrl7,https://api.spotify.com/v1/tracks/3GYlQmwfBDo7...,https://api.spotify.com/v1/audio-analysis/3GYl...,159536,4,Say My Name,"[Dimitri Vegas & Like Mike, Regard, Dimitri Ve...","[73jBynjsVtofjRpdpRAJGk, 4ofCBoyEiGSePFAG500xe...",1
2,0.624,0.628,4,-6.582,0,0.0408,0.0112,0.019100,0.0814,0.772,...,0roOLcll6SSTYZwsQFmXqP,spotify:track:0roOLcll6SSTYZwsQFmXqP,https://api.spotify.com/v1/tracks/0roOLcll6SST...,https://api.spotify.com/v1/audio-analysis/0roO...,147071,4,Let Me Move You - From the Netflix film Work It,[Sabrina Carpenter],[74KM79TiuVKeVCqs8QtB0B],1
3,0.660,0.755,0,-4.501,0,0.0469,0.0876,0.000000,0.1020,0.862,...,72PFP54TZ9Tpj9dYQcb46D,spotify:track:72PFP54TZ9Tpj9dYQcb46D,https://api.spotify.com/v1/tracks/72PFP54TZ9Tp...,https://api.spotify.com/v1/audio-analysis/72PF...,156368,4,NIGHTRIDER,[Arizona Zervas],[0vRvGUQVUjytro0xpb26bs],1
4,0.789,0.711,2,-4.248,1,0.0410,0.0332,0.000000,0.0876,0.753,...,59qrUpoplZxbIZxk6X0Bm3,spotify:track:59qrUpoplZxbIZxk6X0Bm3,https://api.spotify.com/v1/tracks/59qrUpoplZxb...,https://api.spotify.com/v1/audio-analysis/59qr...,190306,4,Take You Dancing,[Jason Derulo],[07YZf4WDAMNwqr4jfgOZ8y],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6678,0.869,0.485,6,-5.595,1,0.0545,0.2460,0.000000,0.0765,0.527,...,6EpRaXYhGOB3fj4V2uDkMJ,spotify:track:6EpRaXYhGOB3fj4V2uDkMJ,https://api.spotify.com/v1/tracks/6EpRaXYhGOB3...,https://api.spotify.com/v1/audio-analysis/6EpR...,204502,4,Strip That Down,"[Liam Payne, Quavo]","[5pUo3fmmHT8bhCyHE52hA6, 0VRj0yCOv2FXJNP47XQnx5]",1
6679,0.728,0.426,0,-6.690,1,0.0509,0.0142,0.000000,0.0546,0.844,...,167NczpNbRF7oWakJaY3Hh,spotify:track:167NczpNbRF7oWakJaY3Hh,https://api.spotify.com/v1/tracks/167NczpNbRF7...,https://api.spotify.com/v1/audio-analysis/167N...,188174,4,Slow Hands,[Niall Horan],[1Hsdzj7Dlq2I7tHP7501T4],1
6680,0.673,0.772,1,-5.246,0,0.0427,0.0223,0.000001,0.1410,0.585,...,6cJqlSO19vsfCsH1FeFI35,spotify:track:6cJqlSO19vsfCsH1FeFI35,https://api.spotify.com/v1/tracks/6cJqlSO19vsf...,https://api.spotify.com/v1/audio-analysis/6cJq...,183200,4,Santeria,[Sublime],[0EdvGhlC1FkGItLOWQzG4J],1
6681,0.584,0.752,8,-6.689,1,0.0580,0.0180,0.000004,0.6470,0.791,...,0zCfmceT1YvtZXGbR6Roht,spotify:track:0zCfmceT1YvtZXGbR6Roht,https://api.spotify.com/v1/tracks/0zCfmceT1Yvt...,https://api.spotify.com/v1/audio-analysis/0zCf...,228173,4,"Another Day Of Sun - From ""La La Land"" Soundtrack",[La La Land Cast],[6T70tWxqyIpdVbsFvPzqM3],1


In [360]:
import pandas as pd 
import spotipy 
from operator import itemgetter

sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
cid ="31d1e524a4c045768a7e34cf4f4d622a" 
secret = "a7676e9d2b45454db896372e6f0986a5" 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False 

playlist_ids = []
playlist_names = []



for i in range(100):
    playlist_results = sp.search(q='sad', type='playlist', limit=1, offset = i)
    playlists = playlist_results["playlists"]["items"] 
    for s in playlists:
        playlist_ids.append((s['id']))
        playlist_names.append((s['name']))
    
#print(playlist_names)
# print(playlist_ids)

count = 0
track_names=[]
playlist_artist_names = []
playlist_artist_ids = []

for p in playlist_ids:
    playlist = sp.playlist(p, fields=all, market=None, additional_types=('track',))
    tracks = playlist["tracks"]["items"] 
    track_ids = []

    #Picking first 100 tracks in a playlist to make it compatible with the audio features limit
    for t in range(100):#len(tracks)):
        try:
            track_ids.append(tracks[t]["track"]["id"]) 
            track_names.append(tracks[t]["track"]["name"])
            track_artists = tracks[t]["track"]["artists"]
            track_artist_names = []
            track_artist_ids = []
            for s in track_artists:
                track_artist_names.append((s['name']))
                track_artist_ids.append((s['id']))

            playlist_artist_names.append(track_artist_names)
            playlist_artist_ids.append(track_artist_ids)
        except:
            pass

    #100 row limit to getting audio features        
    try:
        features=sp.audio_features(track_ids)
    except:
        pass

    if count == 0:
        total_features_df = pd.DataFrame(features)
    else:
        features_df = pd.DataFrame(features)
        total_features_df = total_features_df.append(features_df, ignore_index = True)

    count = count + 1

total_features_df['track_names']=track_names
total_features_df['artist_names'] = playlist_artist_names
total_features_df['artist_ids'] = playlist_artist_ids

sad_df = total_features_df.drop_duplicates(subset = 'id')
sad_df['happy'] = 0
sad_df



/Users/isaacnwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,track_names,artist_names,artist_ids,happy
0,0.556,0.538,8,-5.408,0,0.0382,0.0689,0.000000,0.1960,0.291,...,75ZvA4QfFiZvzhj2xkaWAh,spotify:track:75ZvA4QfFiZvzhj2xkaWAh,https://api.spotify.com/v1/tracks/75ZvA4QfFiZv...,https://api.spotify.com/v1/audio-analysis/75Zv...,223347,4,I Fall Apart,[Post Malone],[246dkjvS1zLTtiykXe5h60],0
1,0.662,0.714,5,-5.680,0,0.1210,0.3120,0.000000,0.1790,0.464,...,52okn5MNA47tk87PeZJLEL,spotify:track:52okn5MNA47tk87PeZJLEL,https://api.spotify.com/v1/tracks/52okn5MNA47t...,https://api.spotify.com/v1/audio-analysis/52ok...,212120,4,Let You Down,[NF],[6fOMl44jA4Sp5b9PpYCkzz],0
2,0.759,0.245,6,-6.545,1,0.1380,0.5150,0.000154,0.1020,0.209,...,4cxMGhkinTocPSVVKWIw0d,spotify:track:4cxMGhkinTocPSVVKWIw0d,https://api.spotify.com/v1/tracks/4cxMGhkinToc...,https://api.spotify.com/v1/audio-analysis/4cxM...,237507,4,Losin Control,[Russ],[1z7b1Pr1rSlvWRzsW3HOrS],0
3,0.397,0.172,6,-15.698,0,0.0445,0.8410,0.000068,0.1080,0.123,...,06UZYUYbMqpaawLgoKwSw9,spotify:track:06UZYUYbMqpaawLgoKwSw9,https://api.spotify.com/v1/tracks/06UZYUYbMqpa...,https://api.spotify.com/v1/audio-analysis/06UZ...,265120,4,Doing It Wrong,[Drake],[3TVXtAsR1Inumwj472S9r4],0
4,0.486,0.204,8,-14.511,1,0.0449,0.9380,0.000000,0.1080,0.243,...,7xSNmARgtbYjg9KqB4QA8P,spotify:track:7xSNmARgtbYjg9KqB4QA8P,https://api.spotify.com/v1/tracks/7xSNmARgtbYj...,https://api.spotify.com/v1/audio-analysis/7xSN...,200745,5,Coaster,[Khalid],[6LuN9FCkKOj5PcnpouEgny],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7603,0.473,0.824,7,-3.680,0,0.1630,0.4820,0.000000,0.6050,0.374,...,3IvMYBE7A3c7to1aEcfFJk,spotify:track:3IvMYBE7A3c7to1aEcfFJk,https://api.spotify.com/v1/tracks/3IvMYBE7A3c7...,https://api.spotify.com/v1/audio-analysis/3IvM...,222929,4,Deep Pockets,[Drake],[3TVXtAsR1Inumwj472S9r4],0
7605,0.458,0.452,9,-9.299,0,0.0470,0.3420,0.000019,0.0703,0.316,...,3Q4gttWQ6hxqWOa3tHoTNi,spotify:track:3Q4gttWQ6hxqWOa3tHoTNi,https://api.spotify.com/v1/tracks/3Q4gttWQ6hxq...,https://api.spotify.com/v1/audio-analysis/3Q4g...,269680,4,Not You Too (feat. Chris Brown),"[Drake, Chris Brown]","[3TVXtAsR1Inumwj472S9r4, 7bXgB6jMjp9ATFy66eO08Z]",0
7606,0.812,0.506,5,-7.706,0,0.1160,0.2810,0.000000,0.3250,0.554,...,7eYAHC0RbBF9eaqWzT34Aq,spotify:track:7eYAHC0RbBF9eaqWzT34Aq,https://api.spotify.com/v1/tracks/7eYAHC0RbBF9...,https://api.spotify.com/v1/audio-analysis/7eYA...,237946,4,Desires (with Future),"[Drake, Future]","[3TVXtAsR1Inumwj472S9r4, 1RyvyyTE3xzB2ZywiAwp0i]",0
7607,0.864,0.477,10,-5.786,1,0.2240,0.2010,0.000000,0.1820,0.500,...,5H4mXWKcicuLKDn4Jy0sK7,spotify:track:5H4mXWKcicuLKDn4Jy0sK7,https://api.spotify.com/v1/tracks/5H4mXWKcicuL...,https://api.spotify.com/v1/audio-analysis/5H4m...,192932,4,Time Flies,[Drake],[3TVXtAsR1Inumwj472S9r4],0


In [369]:
#frames = [happy_df,sad_df]
combined_df = pd.concat([happy_df,sad_df])
combined_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,track_names,artist_names,artist_ids,happy
0,0.813,0.640,2,-5.908,1,0.2720,0.2690,0.000000,0.1310,0.488,...,34FbkFgAbv4ffvhL2XQMVm,spotify:track:34FbkFgAbv4ffvhL2XQMVm,https://api.spotify.com/v1/tracks/34FbkFgAbv4f...,https://api.spotify.com/v1/audio-analysis/34Fb...,147076,4,Jolie nana,[Aya Nakamura],[7IlRNXHjoOCgEAWN5qYksg],1
1,0.729,0.788,10,-4.832,0,0.0908,0.0703,0.000000,0.1150,0.857,...,3GYlQmwfBDo7imFyGPhrl7,spotify:track:3GYlQmwfBDo7imFyGPhrl7,https://api.spotify.com/v1/tracks/3GYlQmwfBDo7...,https://api.spotify.com/v1/audio-analysis/3GYl...,159536,4,Say My Name,"[Dimitri Vegas & Like Mike, Regard, Dimitri Ve...","[73jBynjsVtofjRpdpRAJGk, 4ofCBoyEiGSePFAG500xe...",1
2,0.624,0.628,4,-6.582,0,0.0408,0.0112,0.019100,0.0814,0.772,...,0roOLcll6SSTYZwsQFmXqP,spotify:track:0roOLcll6SSTYZwsQFmXqP,https://api.spotify.com/v1/tracks/0roOLcll6SST...,https://api.spotify.com/v1/audio-analysis/0roO...,147071,4,Let Me Move You - From the Netflix film Work It,[Sabrina Carpenter],[74KM79TiuVKeVCqs8QtB0B],1
3,0.660,0.755,0,-4.501,0,0.0469,0.0876,0.000000,0.1020,0.862,...,72PFP54TZ9Tpj9dYQcb46D,spotify:track:72PFP54TZ9Tpj9dYQcb46D,https://api.spotify.com/v1/tracks/72PFP54TZ9Tp...,https://api.spotify.com/v1/audio-analysis/72PF...,156368,4,NIGHTRIDER,[Arizona Zervas],[0vRvGUQVUjytro0xpb26bs],1
4,0.789,0.711,2,-4.248,1,0.0410,0.0332,0.000000,0.0876,0.753,...,59qrUpoplZxbIZxk6X0Bm3,spotify:track:59qrUpoplZxbIZxk6X0Bm3,https://api.spotify.com/v1/tracks/59qrUpoplZxb...,https://api.spotify.com/v1/audio-analysis/59qr...,190306,4,Take You Dancing,[Jason Derulo],[07YZf4WDAMNwqr4jfgOZ8y],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7603,0.473,0.824,7,-3.680,0,0.1630,0.4820,0.000000,0.6050,0.374,...,3IvMYBE7A3c7to1aEcfFJk,spotify:track:3IvMYBE7A3c7to1aEcfFJk,https://api.spotify.com/v1/tracks/3IvMYBE7A3c7...,https://api.spotify.com/v1/audio-analysis/3IvM...,222929,4,Deep Pockets,[Drake],[3TVXtAsR1Inumwj472S9r4],0
7605,0.458,0.452,9,-9.299,0,0.0470,0.3420,0.000019,0.0703,0.316,...,3Q4gttWQ6hxqWOa3tHoTNi,spotify:track:3Q4gttWQ6hxqWOa3tHoTNi,https://api.spotify.com/v1/tracks/3Q4gttWQ6hxq...,https://api.spotify.com/v1/audio-analysis/3Q4g...,269680,4,Not You Too (feat. Chris Brown),"[Drake, Chris Brown]","[3TVXtAsR1Inumwj472S9r4, 7bXgB6jMjp9ATFy66eO08Z]",0
7606,0.812,0.506,5,-7.706,0,0.1160,0.2810,0.000000,0.3250,0.554,...,7eYAHC0RbBF9eaqWzT34Aq,spotify:track:7eYAHC0RbBF9eaqWzT34Aq,https://api.spotify.com/v1/tracks/7eYAHC0RbBF9...,https://api.spotify.com/v1/audio-analysis/7eYA...,237946,4,Desires (with Future),"[Drake, Future]","[3TVXtAsR1Inumwj472S9r4, 1RyvyyTE3xzB2ZywiAwp0i]",0
7607,0.864,0.477,10,-5.786,1,0.2240,0.2010,0.000000,0.1820,0.500,...,5H4mXWKcicuLKDn4Jy0sK7,spotify:track:5H4mXWKcicuLKDn4Jy0sK7,https://api.spotify.com/v1/tracks/5H4mXWKcicuL...,https://api.spotify.com/v1/audio-analysis/5H4m...,192932,4,Time Flies,[Drake],[3TVXtAsR1Inumwj472S9r4],0


In [399]:
broad_genre = ['pop','country','rock','r&b','hip hop','latin','house','classical','jazz']


def pick_genre(genre_list):
    if not genre_list:
        return None
    genre_count = [0,0,0,0,0,0,0,0,0]
    for i in genre_list:
        for j in range(len(broad_genre)):
            if broad_genre[j] in i:
                genre_count[j] += 1
    if max(genre_count) == 0:
        genre = 'Alternative'
    else:
        genre = broad_genre[genre_count.index(max(genre_count))]
                
                
    return genre

In [405]:
artist_genres = []


for artist_ids in combined_df.artist_ids:
    track_genre = []
    for artist_id in artist_ids:
        #print(artist_id)
        artist = sp.artist(artist_id)
        track_genre = track_genre + artist['genres']
    artist_genres.append(pick_genre(track_genre)) 
        
combined_df['genre'] = artist_genres
combined_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,track_names,artist_names,artist_ids,happy,genre
0,0.813,0.640,2,-5.908,1,0.2720,0.2690,0.000000,0.1310,0.488,...,spotify:track:34FbkFgAbv4ffvhL2XQMVm,https://api.spotify.com/v1/tracks/34FbkFgAbv4f...,https://api.spotify.com/v1/audio-analysis/34Fb...,147076,4,Jolie nana,[Aya Nakamura],[7IlRNXHjoOCgEAWN5qYksg],1,Alternative
1,0.729,0.788,10,-4.832,0,0.0908,0.0703,0.000000,0.1150,0.857,...,spotify:track:3GYlQmwfBDo7imFyGPhrl7,https://api.spotify.com/v1/tracks/3GYlQmwfBDo7...,https://api.spotify.com/v1/audio-analysis/3GYl...,159536,4,Say My Name,"[Dimitri Vegas & Like Mike, Regard, Dimitri Ve...","[73jBynjsVtofjRpdpRAJGk, 4ofCBoyEiGSePFAG500xe...",1,house
2,0.624,0.628,4,-6.582,0,0.0408,0.0112,0.019100,0.0814,0.772,...,spotify:track:0roOLcll6SSTYZwsQFmXqP,https://api.spotify.com/v1/tracks/0roOLcll6SST...,https://api.spotify.com/v1/audio-analysis/0roO...,147071,4,Let Me Move You - From the Netflix film Work It,[Sabrina Carpenter],[74KM79TiuVKeVCqs8QtB0B],1,pop
3,0.660,0.755,0,-4.501,0,0.0469,0.0876,0.000000,0.1020,0.862,...,spotify:track:72PFP54TZ9Tpj9dYQcb46D,https://api.spotify.com/v1/tracks/72PFP54TZ9Tp...,https://api.spotify.com/v1/audio-analysis/72PF...,156368,4,NIGHTRIDER,[Arizona Zervas],[0vRvGUQVUjytro0xpb26bs],1,pop
4,0.789,0.711,2,-4.248,1,0.0410,0.0332,0.000000,0.0876,0.753,...,spotify:track:59qrUpoplZxbIZxk6X0Bm3,https://api.spotify.com/v1/tracks/59qrUpoplZxb...,https://api.spotify.com/v1/audio-analysis/59qr...,190306,4,Take You Dancing,[Jason Derulo],[07YZf4WDAMNwqr4jfgOZ8y],1,pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7603,0.473,0.824,7,-3.680,0,0.1630,0.4820,0.000000,0.6050,0.374,...,spotify:track:3IvMYBE7A3c7to1aEcfFJk,https://api.spotify.com/v1/tracks/3IvMYBE7A3c7...,https://api.spotify.com/v1/audio-analysis/3IvM...,222929,4,Deep Pockets,[Drake],[3TVXtAsR1Inumwj472S9r4],0,pop
7605,0.458,0.452,9,-9.299,0,0.0470,0.3420,0.000019,0.0703,0.316,...,spotify:track:3Q4gttWQ6hxqWOa3tHoTNi,https://api.spotify.com/v1/tracks/3Q4gttWQ6hxq...,https://api.spotify.com/v1/audio-analysis/3Q4g...,269680,4,Not You Too (feat. Chris Brown),"[Drake, Chris Brown]","[3TVXtAsR1Inumwj472S9r4, 7bXgB6jMjp9ATFy66eO08Z]",0,pop
7606,0.812,0.506,5,-7.706,0,0.1160,0.2810,0.000000,0.3250,0.554,...,spotify:track:7eYAHC0RbBF9eaqWzT34Aq,https://api.spotify.com/v1/tracks/7eYAHC0RbBF9...,https://api.spotify.com/v1/audio-analysis/7eYA...,237946,4,Desires (with Future),"[Drake, Future]","[3TVXtAsR1Inumwj472S9r4, 1RyvyyTE3xzB2ZywiAwp0i]",0,hip hop
7607,0.864,0.477,10,-5.786,1,0.2240,0.2010,0.000000,0.1820,0.500,...,spotify:track:5H4mXWKcicuLKDn4Jy0sK7,https://api.spotify.com/v1/tracks/5H4mXWKcicuL...,https://api.spotify.com/v1/audio-analysis/5H4m...,192932,4,Time Flies,[Drake],[3TVXtAsR1Inumwj472S9r4],0,pop


In [408]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10062 entries, 0 to 7608
Data columns (total 23 columns):
danceability        10062 non-null float64
energy              10062 non-null float64
key                 10062 non-null int64
loudness            10062 non-null float64
mode                10062 non-null int64
speechiness         10062 non-null float64
acousticness        10062 non-null float64
instrumentalness    10062 non-null float64
liveness            10062 non-null float64
valence             10062 non-null float64
tempo               10062 non-null float64
type                10062 non-null object
id                  10062 non-null object
uri                 10062 non-null object
track_href          10062 non-null object
analysis_url        10062 non-null object
duration_ms         10062 non-null int64
time_signature      10062 non-null int64
track_names         10062 non-null object
artist_names        10062 non-null object
artist_ids          10062 non-null object
happy

In [410]:
cleaned_df = combined_df.drop(columns = ['type','id','uri','track_href','analysis_url','artist_ids'])

In [411]:
cleaned_df.to_csv('Spotify.csv')